# Physical Space and the Image Grid

### Learning Objectives

* Understand how an **image pixel grid** is defined in **physical space**
* Understand how a **spatial transform** *defines the map between two physical spaces*
* Understand **how to define a resampling operation** to generate a new image grid
* Understand the **role of image interpolators**, and become familiar with a few common interpolators

## Overview

### Physical Space, Images, and the Pixel Grid

When performing image registration, where the object is to find the spatial transform that maps between the fixed image space and the moving image space, it is **extremely important** to have clear definitions of **physical space**, aka **world space**, and how images live in these spaces.

Once we have *good definitions* and *follow them consistently*, operations such resampling, interpolation, multi-resolution processing, and spatial transform as are **clear** and **correct**, even on images that **anisotropic**, **oriented**, or with **multi-component tensor pixels**, such as gradient vectors or diffusion tensors.

We **conceptualize** a **spatial image** as a **uniform grid of pixels** or **pixel grid** that livel is physical space.

Let's continue [with this presentation](https://docs.google.com/presentation/d/1TGjGseDGa_IjrBWA5aMLzxKssy0Al5KT-_FmGlYF92Y/edit?usp=sharing).

### Exercise: Lung image size

There are two 3D CT lung images, sampled on different image grids:

1. *imgs/CT_3D_lung_fixed.mha*
2. *imgs/CT_3D_lung_fixed_small.mha*

Which is larger in terms of its:

* File size?
* Spatial domain?
* Resolution?

How much less time will it take to register *CT_3D_lung_fixed_small.mha* to *CT_3D_lung_moving.mha*?

In [1]:
!ls imgs

CT_3D_lung_fixed.mha	    CT_3D_lung_moving.mha  phantom.tif
CT_3D_lung_fixed_small.mha  phantom2.tif
